## Сравнение тарифов двух и более спонсоров из файлов
- Делается сравнение покрытия по странам
- Делается сравнения покрытия по LTE
- Делается сравнение тарифов по странам с подкраской минимального тарифа (с подкраской LTE)
- Выгружается файл сравнения
- Выполняется проверка пропавших и появившихся операторов и стран для Combined Allowed List
- Готовятся файлы для загрузки тарифов в OCS
- Готовятся файлы для загрузки тарифов в RoamDb

Допонительно:
- Делается сравнение LTE и CAMEL для старого и нового файлов тарифов

По результатам выполнения данной процедуры можно принимать решение об изменении Allowed List.
Для загрузки изменений Allowed List использовать методику c:\W_DATA_ROAM\МЕТОДИКИ\_NOC_M_NEW_CONNECTION\020_DMI_ALLOWED_LIST+STS\

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import datetime as dt
from os.path import join
import roamability as rb

In [2]:
# Из базы данных OCS выгружаем TADIG

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME AS COUNTRY,n.NETWORK_NAME AS NETWORK,t.TADIG_CODE AS TADIG,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_tadig.head(3)

,COUNTRY,NETWORK,TADIG,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4
1,Afghanistan,Areeba/MTN,AFGAR,5
2,Afghanistan,Etisalat,AFG55,6


In [3]:
# Описываем класс данных Спонсора

class SponsorClass:
    
    def __init__(self, name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid,
                 reload_tariff_from_db = False, downloads = None, tariff_file_name = None):
        self.name = name
        self.sponsor_id = sponsor_id
        self.ocs_roaming_plan_id = ocs_roaming_plan_id
        self.roamdb_tarifflistid = roamdb_tarifflistid
        self.df_ocs_cost = DataFrame()
        self.df_roamdb = DataFrame()
        
        self.downloads = downloads
        self.tariff_file_name = tariff_file_name
        
        if reload_tariff_from_db:
            self.upload_data_from_ocs()
            self.upload_data_from_roamdb()
            df_temp = pd.merge(self.df_ocs_cost, self.df_roamdb, how='left', on='TADIG')
            self.df_tariff = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        else:
            self.prepare_pp()
        
    def ocs_tariff_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из OCS'''
        sql_srt=\
        '''
        SELECT t.tadig_code AS TADIG
        ,MO_CALL_RATE MOC,MT_CALL_RATE MTC,MO_SMS_RATE SMS,DATA_RATE DATA
        FROM ROAMING_PLAN_RULES r
        LEFT JOIN RDB_TADIG_CODES t
        ON r.network_id=t.network_id
        LEFT JOIN ROAMING_PLANS p
        ON r.roaming_plan_id=p.roaming_plan_id
        WHERE r.roaming_plan_id IN ({0})
        AND r.SPONSOR_ID = {1}
        AND p.deleted IS NULL
        AND r.end_date IS NULL
        AND r.is_discounted = 1
        '''.format(self.ocs_roaming_plan_id, self.sponsor_id)
        return sql_srt
    
    def roamdb_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из RoamDB'''
        sql_srt=\
        '''
        SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
        ,t.dataincrement_KB AS Data_Increment_KB
        ,IIF(t.support4G = 1, 'Yes', 'No') AS LTE,tc.value AS TADIG FROM Tariffs t
        INNER JOIN TadigCodes tc
        ON t.OperatorID = tc.OperatorID
        WHERE t.tarifflistid = {0}
        '''.format(self.roamdb_tarifflistid)
        return sql_srt
    
    def upload_data_from_ocs(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
        sql_srt = self.ocs_tariff_gen()
        self.df_ocs_cost = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def upload_data_from_roamdb(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')
        sql_srt = self.roamdb_gen()
        self.df_roamdb = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def prepare_pp(self):
        '''Функция для подготовки DF из файла.
        Объединение с TADIG, COUNTRY и NETWORK_ID из OCS.
        Фильтрация по максимальному тарифу передачи данных.'''
        df_temp = pd.read_csv(join(self.downloads, self.tariff_file_name))
        df_temp = df_temp[df_temp['DATA'] <= discount_data_price_threshold]
        df_temp = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        self.df_tariff = df_temp

In [17]:
#######################################################################################
# Define variables here
#######################################################################################

dmi_allowed_list = 'DMI.dmi_allowed_list_export_Sun_Dec_20_2020.csv'

combined_dict = {'Partner_Combined':'S1', 'P4_Combined':'S2', 'MB_Combined':'S4',
                 'SMART_Combined':'S5','S6_Combined':'S6','S8_Combined':'S8',
                 'S9_Combined':'S9','S10_Combined':'S10'}

discount_data_price_threshold = 0.7 # USD
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'

reload_tariff_from_db = False # False if the tariffs should be taken from files

# Output file names
file_compare_by_country_coverage = 'CompareByCountryCoverage.csv'
file_compare_by_lte_coverage = 'CompareByLteCoverage.csv'
file_compare_tariffs = 'CompareTariffs.csv'
file_country_min_tariffs = 'CountryMinTariff.xlsx'
file_country_min_tariffs_lte = 'CountryMinTariffLte.xlsx'
file_operator_min_tariffs_lte = 'OperatorMinTariffLte.xlsx'

# Files with the tariffs should contain the following fields:
# TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE

# name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid, reload_tariff_from_db, downloads, tariff_file_name
s1_sponsor = SponsorClass('S1', 1, 267, 41, tariff_file_name = 'S1_20210120_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s2_sponsor = SponsorClass('S2', 2, 268, 40, tariff_file_name = 'S2_20210101_FULL_USD.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s4_sponsor = SponsorClass('S4', 4, 329, 36, tariff_file_name = 'S4_20200801_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s5_sponsor = SponsorClass('S5', 5, 343, 32, tariff_file_name = 'S5_20200616_FULL_RB_COST.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s6_sponsor = SponsorClass('S6', 6, 368, 39, tariff_file_name = 'S6_20200818_FULL_USD.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s8_sponsor = SponsorClass('S8', 8, 372, 42, tariff_file_name = 'S8_20210112_FULL_USD.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s9_sponsor = SponsorClass('S9', 9, 999, 99, tariff_file_name = 'S9_20200601_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s10_sponsor = SponsorClass('S10', 10, 999, 99, tariff_file_name = 'S10_20201215_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)

sp01_sponsor = SponsorClass('SP01', 101, 999, 99, tariff_file_name = 'SP01_20210201_FULL_USD.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
sp02_sponsor = SponsorClass('SP02', 102, 999, 99, tariff_file_name = 'SP02_20210201_FULL_USD.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)

# sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor, s10_sponsor,
#            sp01_sponsor, sp02_sponsor]

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor, s10_sponsor,
           sp01_sponsor, sp02_sponsor]

#######################################################################################

#######################################################################################

In [18]:
for sponsor in sponsors:
    print(f'\nTariffs of {sponsor.name}:')
    display(sponsor.df_tariff.head(3))


Tariffs of S1:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBVF,0.03,0.0,0.006,0.0024,1,1.0,Yes,Yes,NaN,NaN,Albania,Vodafone,12
1,ALBAM,0.26,0.0,0.030,0.0090,1,1.0,No,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9
2,ANDMA,0.09,0.0,0.030,0.0300,1,1.0,No,No,NaN,NaN,Andorra,Andorra Telecom S A U,17



Tariffs of S2:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBAM,0.51635,0,0.05980,0.04600,1,1,Yes,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9
1,ALBEM,0.44850,0,0.04600,0.04485,60,1,Yes,Yes,NaN,NaN,Albania,Eagle Mobile,10
2,ARGTM,NaN,0,0.16445,0.04600,60,1,No,Yes,NaN,NaN,Argentina,Movistar/Telefonica,27



Tariffs of S4:


...

,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBM4,0.12,0,0.015,0.012,NaN,NaN,No,No,NaN,NaN,Albania,PLUS Communication Sh.a,11
1,ALBVF,0.12,0,0.015,0.012,NaN,NaN,Yes,Yes,NaN,NaN,Albania,Vodafone,12
2,ALBAM,0.12,0,0.015,0.012,NaN,NaN,Yes,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9



Tariffs of SP01:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,AIAK9,NaN,0.073473,0.062606,0.05750,60.0,1,NaN,No,NaN,NaN,NaN,NaN,NaN
1,ALBAM,0.6325,NaN,0.306671,0.13417,NaN,50,NaN,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9
2,ALGOT,NaN,NaN,0.197674,0.57500,NaN,100,NaN,No,NaN,NaN,NaN,NaN,NaN



Tariffs of SP02:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBAM,0.421671,0.0,0.140553,0.025553,60,1,NaN,Yes,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9
1,ALBVF,0.285246,0.0,0.014260,0.057500,60,1,NaN,Yes,NaN,NaN,Albania,Vodafone,12
2,ANDMA,0.070276,0.0,0.028106,0.153330,60,1,NaN,Yes,NaN,NaN,Andorra,Andorra Telecom S A U,17


In [19]:
# Загрузить Combined Allowed list

df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_AL','Sponsor':'SPONSOR'}, inplace=True, axis=1)
df_al = df_al[df_al.SPONSOR.isin(list(combined_dict))]
df_al = df_al.replace(combined_dict)
df_al = pd.merge(df_al, df_tadig[['COUNTRY','TADIG','NETWORK_ID']], how='left', left_on='TADIG_AL', right_on='TADIG')
df_al.drop('TADIG', inplace=True,axis=1)

# Проверить дублирования стран в Combined Allowed List

s_temp = df_al.groupby('COUNTRY')['SPONSOR'].nunique()
if s_temp[s_temp > 1].count():
    print('Duplicates per Country in the Combined Allowed List:')
    display(s_temp[s_temp > 1])
else:
    print('No duplicates per Country found in the Combined Allowed List.')

df_al.head(3)

No duplicates per Country found in the Combined Allowed List.


,SPONSOR,TADIG_AL,COUNTRY,NETWORK_ID
0,S2,HRVT2,Croatia,225
1,S2,CZEET,Czech Republic,234
2,S1,GBRAJ,United Kingdom,998


In [14]:
# Получаем данные по Usage из BSS

start_date = '20201201'
end_date = '20201231'

sql_srt="""
SELECT RepDate,SponsorPrefix,VisitedNetworkId AS NETWORK_ID,UsageType,SUM(Cost) Cost, SUM(Revenue) Revenue, SUM(Volume) Volume FROM
(
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,Volume
,'DATA' AS UsageType
FROM aggreg.GPRS_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,NbEvent AS Volume
,CASE WHEN Sms = 0 THEN 'MOC' WHEN Sms = 1 THEN 'MOSMS' END AS UsageType
FROM aggreg.MOC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,NbEvent AS Volume
,CASE WHEN Sms = 0 THEN 'MTC' WHEN Sms = 1 THEN 'MTSMS' END AS UsageType
FROM aggreg.MTC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
) t GROUP BY RepDate,SponsorPrefix,VisitedNetworkId,UsageType
""".format(start_date=start_date, end_date=end_date)

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_source = pd.read_sql_query(sql_srt, cnxn)

# sql_srt='SELECT network_id AS VisitedNetworkId, MAX(tadig_code) AS Tadig FROM RDB_TADIG_CODES GROUP BY network_id;'
# with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
#     df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_bss_source.SponsorPrefix.replace({42501:1, 26006:2, 45403:4, 51503:5}, inplace=True)
df_bss_source = df_bss_source.pivot_table(values='Volume',index=['RepDate','SponsorPrefix','NETWORK_ID']
                   ,columns='UsageType', aggfunc=np.sum).reset_index()

df_bss_processed = pd.merge(df_bss_source, df_tadig[['COUNTRY','NETWORK_ID']].drop_duplicates(), how='left', on='NETWORK_ID')
df_bss_processed = df_bss_processed[['RepDate','SponsorPrefix','COUNTRY','NETWORK_ID','MOC','MTC','MOSMS','MTSMS','DATA']]
df_bss_processed.DATA = df_bss_processed.DATA/1024/1024/1024
df_bss_processed.head(3)

,RepDate,SponsorPrefix,COUNTRY,NETWORK_ID,MOC,MTC,MOSMS,MTSMS,DATA
0,2020-12,1,Albania,12,308.0,16.0,4.0,135.0,10.139857
1,2020-12,1,Antigua and Barbuda,23,NaN,NaN,NaN,NaN,0.209677
2,2020-12,1,Argentina,27,5.0,1.0,3.0,101.0,13.723597


### Cравнение покрытия по странам

In [11]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'DATA_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')

df = df[df.isnull().any(axis=1)]
df.to_csv(join(downloads, file_compare_by_country_coverage), index=False)
df.head()

,COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S6,DATA_S8,DATA_S9
0,Albania,0.0024,0.04485,NaN,0.008625,0.01495,0.0204,NaN
1,Andorra,0.0300,NaN,NaN,NaN,NaN,NaN,NaN
2,Anguilla,0.0257,NaN,NaN,NaN,0.17940,0.1260,NaN
3,Antigua and Barbuda,0.0240,NaN,NaN,NaN,0.17940,0.1260,NaN
4,Argentina,0.0140,0.04600,NaN,0.009200,0.02093,0.0540,NaN


### Сравнение покрытия по LTE

In [13]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['LTE'].max().rename({'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')

df = df[(df == 'Yes').any(axis=1) & (df == 'No').any(axis=1)]
df.to_csv(join(downloads, file_compare_by_lte_coverage), index=False)
df.head()

,COUNTRY,LTE_S1,LTE_S2,LTE_S4,LTE_S5,LTE_S6,LTE_S8,LTE_S9
0,Albania,Yes,Yes,NaN,No,Yes,Yes,NaN
2,Anguilla,No,NaN,NaN,NaN,No,Yes,NaN
3,Antigua and Barbuda,No,NaN,NaN,NaN,No,Yes,NaN
4,Argentina,Yes,No,NaN,Yes,No,Yes,NaN
5,Armenia,No,NaN,NaN,No,Yes,Yes,NaN


### Сравнение тарифов по странам

In [9]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'DATA_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')
    
df = pd.merge(df, df_bss_processed.groupby('COUNTRY', as_index=False)['DATA'].sum(), how='left', on='COUNTRY')

df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_country_compare = df.sort_values(by='DATA', ascending=False)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #81EF21' if v else '' for v in is_max]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

df_country_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
.to_excel(join(downloads, file_country_min_tariffs), 'Sheet1', index=False)

display(df_country_compare.head(20).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1).hide_index())

COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S6,DATA_S8,DATA,SPONSOR
United States,nan,0.00345,0.00504,0.00575,0.00138,0.018,2284.06,S2
Turkey,0.0024,0.00322,0.00504,0.00862,0.00644,0.012,1407.34,S1
United Kingdom,0.0016,0.00138,nan,0.00575,0.00184,0.0048,1403.52,S1
Canada,0.005,0.02875,0.00504,0.00345,0.00264,0.018,996.709,S1
Russian Federation,0.004,0.0345,0.01008,0.0115,0.00805,0.018,772.096,S1
Italy,0.0018,0.00299,nan,0.00862,0.00196,0.0048,719.237,S1
Germany,0.0024,0.00121,nan,0.00862,0.0015,0.0072,503.19,S2
Sweden,0.0016,0.0015,nan,0.00236,0.00092,0.0024,495.045,S2
Spain,0.0024,0.00207,nan,0.00862,0.00184,0.012,436.242,S2
Australia,0.0022,0.0092,0.00504,0.0023,0.00897,0.0072,395.978,S1


### Новые страны для Combined Allowed List

In [10]:
# Новые страны для Combined Allowed List

df_country_compare[df_country_compare.SPONSOR.isna()]

,COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S6,DATA_S8,DATA,SPONSOR
42,Gibraltar,0.0120,0.004600,NaN,0.011500,NaN,0.0120,2.741275,NaN
156,Faroe Islands,NaN,NaN,NaN,NaN,0.005290,0.0144,1.197800,NaN
166,Bolivia,NaN,NaN,NaN,NaN,NaN,0.0432,1.070367,NaN
163,Tunisia,NaN,NaN,NaN,NaN,0.007475,0.0432,0.048890,NaN
127,Iran,NaN,0.066125,NaN,NaN,NaN,0.0180,0.003523,NaN
54,Ivory Coast,0.0566,NaN,NaN,0.011500,0.014950,0.1044,0.000220,NaN
122,Venezuela,0.2000,NaN,NaN,NaN,NaN,0.0528,0.000099,NaN
153,Algeria,NaN,NaN,NaN,NaN,0.007475,0.0228,0.000041,NaN
161,Palestinian Territory,NaN,NaN,NaN,NaN,0.074750,NaN,0.000019,NaN
1,Andorra,0.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Пропавшие и новые сети для Combined Allowed List

In [11]:
fields_list = ['COUNTRY','TADIG','TADIG_AL','DATA','SPONSOR']
df_new_mis_operatirs = DataFrame()

for sponsor in sponsors:
    df_sponsor = pd.merge(sponsor.df_tariff, df_al[df_al.SPONSOR == sponsor.name],
                       how='outer', on=['COUNTRY','NETWORK_ID'])[fields_list]
    s_sponsor = df_sponsor.loc[df_sponsor.SPONSOR == sponsor.name, 'COUNTRY'].unique()
    df_sponsor = df_sponsor.loc[df_sponsor.COUNTRY.isin(s_sponsor)]
    df_mis = df_sponsor.loc[df_sponsor.TADIG.isna()].copy()
    df_mis['COM'] = 'MIS'
    df_new = df_sponsor.loc[df_sponsor.SPONSOR.isna()].copy()
    df_new['COM'] = 'NEW'
    df_new['SPONSOR'] = sponsor.name
    df_new_mis_operatirs = df_new_mis_operatirs.append(df_mis).append(df_new)
    
df_new_mis_operatirs.sort_values(by='COUNTRY')

,COUNTRY,TADIG,TADIG_AL,DATA,SPONSOR,COM
1,Afghanistan,AFGAR,NaN,0.104400,S8,NEW
1,Albania,ALBAM,NaN,0.009000,S1,NEW
6,Argentina,ARGCM,NaN,0.025000,S1,NEW
7,Argentina,ARGTP,NaN,0.025000,S1,NEW
41,Bangladesh,BGDGP,NaN,0.020400,S8,NEW
27,Brazil,BRACL,NaN,0.025000,S1,NEW
39,Chile,CHLSM,NaN,0.025000,S1,NEW
58,Germany,DEUE1,NaN,0.001208,S2,NEW
71,Iceland,ISLTL,NaN,0.004600,S2,NEW
94,India,INDIB,NaN,0.020000,S1,NEW


### Выгрузка полного сравнительного листа в файл

In [17]:
# Выгрузка полного сравнительного листа в файл

df = DataFrame({'COUNTRY':[],'NETWORK_ID':[],'NETWORK':[],'TADIG':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.rename({'MOC':f'MOC_{sponsor.name}'
                                        ,'MTC':f'MTC_{sponsor.name}'
                                        ,'SMS':f'SMS_{sponsor.name}'
                                        ,'DATA':f'DATA_{sponsor.name}'
                                        ,'LTE':f'LTE_{sponsor.name}'}
                                        , axis=1).drop(['Voice_Increment_SEC','Data_Increment_KB'], axis=1)
    df = pd.merge(df, df_temp, how='outer', on=['COUNTRY','NETWORK_ID','NETWORK','TADIG'])
    
cols = ['COUNTRY','NETWORK','TADIG']
for sponsor in sponsors:
    cols.extend([f'MOC_{sponsor.name}',f'MTC_{sponsor.name}',f'SMS_{sponsor.name}',f'DATA_{sponsor.name}',f'LTE_{sponsor.name}'])

df[cols].to_csv(join(downloads, file_compare_tariffs), index=False)
df[cols].head()

,COUNTRY,NETWORK,TADIG,MOC_S1,MTC_S1,SMS_S1,DATA_S1,LTE_S1,MOC_S2,MTC_S2,...,MOC_S8,MTC_S8,SMS_S8,DATA_S8,LTE_S8,MOC_S9,MTC_S9,SMS_S9,DATA_S9,LTE_S9
0,Albania,Vodafone,ALBVF,0.0300,0.0,0.0060,0.0024,Yes,NaN,NaN,...,0.2460,0.0000,0.1020,0.0204,Yes,NaN,NaN,NaN,NaN,NaN
1,Andorra,Andorra Telecom S A U,ANDMA,0.0900,0.0,0.0300,0.0300,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,No,NaN,NaN,...,0.3576,0.0000,0.2388,0.1260,Yes,NaN,NaN,NaN,NaN,NaN
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,No,NaN,NaN,...,0.3576,0.0000,0.2388,0.1260,Yes,NaN,NaN,NaN,NaN,NaN
4,Argentina,Movistar/Telefonica,ARGTM,0.0600,0.0,0.0150,0.0140,Yes,NaN,0.0,...,1.6020,0.2676,0.3576,0.0540,Yes,NaN,NaN,NaN,NaN,NaN


### Сравнение тарифов по странам с подкраской LTE

In [20]:
# Подготовка DF для сравнения тарифов по странам с учетом LTE

df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.loc[:, ['COUNTRY', 'MOC', 'DATA', 'LTE']]
    df_temp = df_temp.loc[df_temp.groupby('COUNTRY')['DATA'].idxmin()]\
    .rename({'MOC':f'MOC_{sponsor.name}', 'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on='COUNTRY')

df = pd.merge(df, df_bss_processed.groupby('COUNTRY', as_index=False)['DATA'].sum(), how='left', on='COUNTRY')

df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_country_compare = df.sort_values(by='DATA', ascending=False)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_country_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_country_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_country_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,MOC_S1,DATA_S1,LTE_S1,MOC_S2,DATA_S2,LTE_S2,MOC_S4,DATA_S4,LTE_S4,MOC_S5,DATA_S5,LTE_S5,MOC_S6,DATA_S6,LTE_S6,MOC_S8,DATA_S8,LTE_S8,MOC_S10,DATA_S10,LTE_S10,MOC_SP01,DATA_SP01,LTE_SP01,MOC_SP02,DATA_SP02,LTE_SP02,DATA,SPONSOR
United States,nan,nan,nan,0.023,0.00345,Yes,0.26,0.00504,Yes,0.0345,0.00575,Yes,0.02093,0.00138,Yes,nan,0.01265,Yes,0.12,0.0035,Yes,nan,0.00255,nan,0.03496,0.00383,nan,3109.43,S2
United Kingdom,0.0313,0.0016,Yes,0.0207,0.00138,Yes,nan,nan,nan,0.1518,0.00575,No,0.02242,0.00184,Yes,17.8434,0.0023,Yes,0.12,0.0015,Yes,0.05239,0.00383,nan,0.04089,0.00255,nan,1530.31,S1
Canada,0.045,0.005,Yes,0.0828,0.02875,Yes,0.26,0.00504,Yes,0.23,0.00345,Yes,0.1196,0.00264,Yes,nan,0.0161,Yes,0.12,0.0035,Yes,0.08945,0.01405,nan,0.06821,0.00767,nan,1498.74,S1
Italy,0.0313,0.0018,No,0.0207,0.00299,No,nan,nan,nan,0.161,0.00862,Yes,0.01909,0.00196,Yes,0.6256,0.0046,Yes,0.12,0.0014,Yes,0.04983,0.00422,nan,0.04089,0.00255,nan,1145,S1
United Arab Emirates,0.25,0.007,Yes,nan,nan,nan,0.52,0.01197,Yes,0.2875,0.0575,Yes,0.63687,0.02726,Yes,2.46675,0.0161,Yes,0.2,0.007,Yes,nan,nan,nan,0.575,0.19167,nan,1052.63,S1
Russian Federation,0.025,0.004,Yes,0.23,0.0345,Yes,0.26,0.01008,Yes,0.23,0.0115,Yes,0.52106,0.00805,Yes,0.0989,0.0092,Yes,0.25,0.0035,Yes,0.32583,0.00255,nan,0.26833,0.00125,nan,893.506,S1
Turkey,0.03,0.0024,Yes,0.2093,0.00322,Yes,0.26,0.00504,Yes,0.161,0.00862,Yes,0.2093,0.00644,Yes,0.41975,0.0092,No,0.25,0.0022,Yes,0.21083,0.00383,nan,0.38333,0.00639,nan,828.821,S1
Sweden,0.0313,0.0016,No,0.023,0.0015,Yes,nan,nan,nan,0.299,0.00236,No,0.013,0.00092,Yes,nan,0.0069,Yes,0.12,0.0025,Yes,0.01278,0.00128,nan,0.04089,0.00255,nan,610.303,S2
Germany,0.03,0.0024,Yes,0.01955,0.00121,No,nan,nan,nan,0.161,0.00862,Yes,0.01852,0.0015,Yes,0.2185,0.0069,Yes,0.12,0.004,No,nan,0.00383,nan,0.04217,0.0023,nan,572.111,S2
Tanzania,0.03,0.0024,Yes,0.57684,0.35822,No,nan,nan,nan,nan,nan,nan,0.3887,0.13455,No,nan,0.0092,Yes,nan,nan,nan,0.74635,0.33925,nan,0.1181,0.0575,nan,503.686,S1


### Сравнение тарифов по операторам с подкраской LTE

In [21]:
# Сравнение тарифов по операторам с подкраской LTE

df = DataFrame({'COUNTRY':[], 'NETWORK':[], 'TADIG':[],'NETWORK_ID':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby(['COUNTRY','NETWORK','NETWORK_ID'],
                                        as_index=False)[['TADIG', 'MOC', 'DATA', 'LTE']].first()\
    .rename({'MOC':f'MOC_{sponsor.name}', 'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on=['COUNTRY','TADIG','NETWORK','NETWORK_ID'])

# df = pd.merge(df, df_bss_processed[['NETWORK_ID','DATA']], how='left', on='NETWORK_ID')
df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_operator_compare = df.sort_values(by='COUNTRY', ascending=True)
df_operator_compare.drop(['NETWORK_ID'], axis=1, inplace=True)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_operator_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_operator_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_operator_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,NETWORK,TADIG,MOC_S1,DATA_S1,LTE_S1,MOC_S2,DATA_S2,LTE_S2,MOC_S4,DATA_S4,LTE_S4,MOC_S5,DATA_S5,LTE_S5,MOC_S6,DATA_S6,LTE_S6,MOC_S8,DATA_S8,LTE_S8,MOC_S10,DATA_S10,LTE_S10,MOC_SP01,DATA_SP01,LTE_SP01,MOC_SP02,DATA_SP02,LTE_SP02,SPONSOR
Afghanistan,Afghan Wireless Communication Company,AFGAW,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.115,No,0.37375,0.07475,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,S8
Afghanistan,Areeba/MTN,AFGAR,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0828,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,S8
Afghanistan,Etisalat,AFGEA,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.01725,No,nan,nan,nan,0.6325,0.23575,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,S8
Albania,TELEKOM ALBANIA SH A,ALBAM,0.26,0.009,No,0.51635,0.046,Yes,nan,nan,nan,nan,nan,nan,0.4485,0.01495,Yes,nan,0.01495,Yes,0.12,0.012,Yes,0.6325,0.13417,nan,0.42167,0.02555,nan,S1
Albania,Vodafone,ALBVF,0.03,0.0024,Yes,nan,nan,nan,nan,nan,nan,0.161,0.00862,No,0.22425,0.01495,Yes,0.1955,0.0115,Yes,0.12,0.012,Yes,nan,nan,nan,0.28525,0.0575,nan,S1
Albania,Eagle Mobile,ALBEM,nan,nan,nan,0.4485,0.04485,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.2438,0.0506,Yes,0.12,0.012,Yes,nan,nan,nan,nan,nan,nan,S1
Albania,PLUS Communication Sh.a,ALBM4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.12,0.012,No,nan,nan,nan,nan,nan,nan,S1
Algeria,ATM Mobils,DZAA1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.299,0.00748,No,nan,0.01265,Yes,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Algeria,Oreedo/Wataniya / Nedjma,DZAWT,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.37375,0.03738,No,nan,0.0184,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Andorra,Andorra Telecom S A U,ANDMA,0.09,0.03,No,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.01917,nan,0.07028,0.15333,nan,nan


### Подготовить файлы со всеми тарифами из файла для загрузки в OCS

In [20]:
# Подготовить файлы со всеми тарифами для загрузки в OCS

tariff_start_date = '2020-01-01 00:00:00'
charge_cost_increment = 2

def create_file_upload_to_ocs(sponsor_obj):
    df = sponsor_obj.df_tariff.copy()
    df['ROAMING_PLAN_RULE_ID'] = ''
    df['ROAMING_PLAN_ID']=''
    df['SPONSOR_ID'] = sponsor_obj.sponsor_id
    df['MT_SMS_RATE'] = 0
    df['CURRENCY_ID'] = 1 #1:USD, 2:EUR
    df['START_DATE'] = tariff_start_date
    df['END_DATE'] = 'NULL'
    df['is_discounted'] = 1
    return df

for sponsor in sponsors:
    df_temp = create_file_upload_to_ocs(sponsor)
    df_temp[['ROAMING_PLAN_RULE_ID',
            'ROAMING_PLAN_ID',
            'NETWORK_ID',
            'SPONSOR_ID',
            'MOC',
            'MTC',
            'SMS',
            'MT_SMS_RATE',
            'DATA',
            'CURRENCY_ID',
            'START_DATE',
            'END_DATE',
            'is_discounted']].drop_duplicates().to_csv(join(downloads, f'ocs_insert_{sponsor.name}_cost.csv'), index=False)
    
    df_temp[['MOC','MTC','SMS','DATA']] = df_temp[['MOC','MTC','SMS','DATA']] * charge_cost_increment
    
    df_temp[['ROAMING_PLAN_RULE_ID',
            'ROAMING_PLAN_ID',
            'NETWORK_ID',
            'SPONSOR_ID',
            'MOC',
            'MTC',
            'SMS',
            'MT_SMS_RATE',
            'DATA',
            'CURRENCY_ID',
            'START_DATE',
            'END_DATE',
            'is_discounted']].drop_duplicates().to_csv(join(downloads, f'ocs_insert_{sponsor.name}_charge.csv'), index=False)

### Подготовить файлы со всеми тарифами из файла для загрузки в RoamDB

In [13]:
# Подготовить файлы со всеми тарифами для загрузки в RoamDB

columns_origin = ['TADIG',
  'MOC',
  'MTC',
  'SMS',
  'DATA',
  'Voice_Increment_SEC',
  'Data_Increment_KB',
  'IsDiscounted (Yes/No)',
  'LTE']

columns_roamdb = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

dict_roamdb_columns = dict(zip(columns_origin,columns_roamdb))

def create_file_upload_to_roamdb(sponsor_obj):
    df = sponsor_obj.df_tariff.copy()
    df['IsDiscounted (Yes/No)'] = 'Yes'
    return df.rename(dict_roamdb_columns, axis=1)

for sponsor in sponsors:
    df_temp = create_file_upload_to_roamdb(sponsor)
    df_temp[columns_roamdb].drop_duplicates().to_csv(join(downloads, f'roamdb_upload_{sponsor.name}.csv'), index=False)

Для загрузки в RoamDb удалить все тарифы спонсора из тарифа

USE [RoamDb];
GO

SELECT * FROM sys.tables ORDER BY name

SELECT * FROM TariffLists
SELECT * FROM Tariffs

BEGIN TRAN

DELETE FROM Tariffs WHERE TariffListID = 

--COMMIT TRAN
--ROLLBACK TRAN

SELECT * FROM Tariffs WHERE TariffListID = 

### Сравнение LTE и CAMEL для старого и нового файлов тарифов

In [ ]:
file_s2_old = 'S2_20200601_FULL_USD.csv'
file_s2_new = 'S2_20200701_FULL_USD.csv'
df_s2_old = pd.read_csv(join(downloads, file_s2_old), usecols=['TADIG','LTE','CAMEL'])
df_s2_new = pd.read_csv(join(downloads, file_s2_new), usecols=['TADIG','LTE','CAMEL'])
df_temp = pd.merge(df_s2_old, df_s2_new, how='outer', on='TADIG', suffixes=['_OLD', '_NEW'])
df_temp.head()
print('Отличия LTE:')
display(df_temp.loc[(df_temp.LTE_OLD != df_temp.LTE_NEW), ['TADIG', 'LTE_OLD', 'LTE_NEW']])
print('\nОтличия CAMEL:')
display(df_temp.loc[(df_temp.CAMEL_OLD != df_temp.CAMEL_NEW), ['TADIG', 'CAMEL_OLD', 'CAMEL_NEW']])

### Конвертация EUR в USD

In [10]:
eur_usd_rate = 1.15

tariff_file_name_eur = 'SP02_20210201_FULL_EUR.csv'
tariff_file_name_usd = 'SP02_20210201_FULL_USD.csv'

df_temp = pd.read_csv(join(downloads, tariff_file_name_eur))
df_temp[['MOC','MTC','SMS','DATA']] = df_temp[['MOC','MTC','SMS','DATA']] * eur_usd_rate
df_temp.to_csv(join(downloads, tariff_file_name_usd), index=False)
df_temp.head()

,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit
0,AFGAR,3.149114,0.524848,0.524848,9.433887,60,10,NaN,Yes,NaN,NaN
1,AFGK9,3.247853,0.649566,0.649566,10.405901,60,10,NaN,Yes,NaN,NaN
2,AFGAW,2.598287,0.649566,0.649566,10.405901,60,10,NaN,Yes,NaN,NaN
3,AFGEA,3.401378,0.566904,0.566904,11.350730,60,10,NaN,Yes,NaN,NaN
4,ALBAM,0.421670,0.000000,0.140553,0.025553,60,1,NaN,Yes,NaN,NaN


### Разработки